In [ ]:
import numpy as np
import pickle
import pandas as pd

# G-code generation codes are cloned from https://github.com/tibor-barsi/GcodeGenerator. Tibor Barsi is the author of the code, and was a PhD student at the Ladisk lab of the University of Ljubljana
#  We should be careful with crediting the author of the code, if we ever want to make these code public.
# G-code generation codes are cloned from https://github.com/tibor-barsi/GcodeGenerator. Tibor Barsi is the author of the code.
from src.g_code_generation_copy.gcode_generator import G_code_generator
from src.g_code_generation_copy.tool_changer_functions import load_params
from src.printing import *
from src.network import Network_custom
import os

In [ ]:
import numpy as np

def generate_expandable_octahedron(scale=1.0, q_cable=1.0, q_strut=-1.0):
    """
    Generate vertices, edges, force densities, and fixed vertices for an expandable octahedron tensegrity structure.
    
    Parameters:
    scale   : float - Scaling factor for output coordinates
    q_cable : float - Force density for cables (positive)
    q_strut : float - Force density for struts (negative)
    
    Returns:
    vertices       : np.array - Array of vertex coordinates
    edges         : list    - List of edges connecting vertex indices
    force_densities : list - List of force densities for each edge
    fixed         : list    - List of fixed vertex indices
    """
    # Define vertex coordinates 
    vertices = np.array([
        [-0.5, 0, -1],  # 1 
        [-0.5, 0, 1],  # 2
        [0.5, 0, -1],   # 3  #old: 
        [0.5, 0, 1],   # 4 
        [0, 1, -0.5],  # 5
        [0, -1, -0.5],    # 6  #old: 
        [-1, 0.5, 0],  # 7
        [1, 0.5, 0],   # 8
        [-1, -0.5, 0],  # 9
        [1, -0.5, 0],   # 10
        [0, 1, 0.5],  # 11
        [0, -1, 0.5]    # 12
        ]) * scale
    
    # Define edges
    # edges_cables = [[1, 7], [1, 9], [5, 1], [1, 6], [3,5], [6, 3], [3, 8], [3, 10], [5, 7], [5, 8], [6,9], [6, 10], [2, 7], [7, 11], [9, 2], [9, 12], 
            #  [8, 11], [4, 8], [10, 12], [4, 10], [2, 11], [11, 4], [12, 2], [4,12]
    # ]
    edges_cables = [[7, 1], [1, 9], [1, 5], [1, 6], [3,5], [3,6], [3, 8], [3, 10], [5, 7], [5, 8], [6,9], [6, 10], [2, 7], [7, 11], [2, 9], [9, 12], 
             [11, 8], [4, 8], [10, 12], [4, 10], [2, 11], [4, 11], [2, 12], [4,12]
    ]
    edges_struts = [[1,2], [3,4], [5, 6], [7, 8], [9, 10], [11, 12]]
    edges = edges_cables + edges_struts
    
    edges = [[i-1, j-1] for i, j in edges]
    cable_slice = slice(0, len(edges_cables))
    strut_slice = slice(len(edges_cables), len(edges_cables) + len(edges_struts))
    # Assign force densities
    force_densities = [q_cable] * len(edges_cables) + [q_strut] * len(edges_struts)
    
    # Fix the bottom vertex to prevent free movement
    fixed = [0, 2, 5]#[1, 3, 6]
    # fixed = [0, 1, 4]#[1, 3, 6]
    
    return vertices, edges, force_densities, fixed, cable_slice, strut_slice
model_name = 'expandable_octahedron'
# Example usage
q_scalar = 0.012
vertices, edges, force_densities, fixed, cable_slice, strut_slice = generate_expandable_octahedron(scale=40, q_cable=1*q_scalar, q_strut=-1.5*q_scalar)

paths = [[3, 5, 4, 2, 1, 14, 20,21,23,22], [0, 13, 16,6, 7, 18,15], [10,11,19,17,9,8,12]]
directions = np.ones(len(edges))

# net = Network_custom.from_fd(vertices, edges, force_densities, fixed, paths = None, dir = None)
net = Network_custom.direct(vertices, edges, force_densities, fixed, paths = paths, dir = directions)
net.net_plot(color = True,vlabels = True, elables=False, path_colors=True)

Force densities, and length ratios as reported in the paper

In [ ]:
net.q, net.l1[-1]/net.l1[0]

Flatten the network

In [ ]:
center = np.mean(net.vertices, axis=0)  # Center of the point cloud
axis = np.array([0, 0, 1])  # Example: Unwrapping around Z-axis

# Choose one of three methods to flatten a network. (In this example Projection is not appropriate, but is added for completeness)
method = "radial"
kwargs = {'axis': axis}
vertices_flattend = net.flatten_network(method = method, **kwargs)

# method = "spherical"
# kwargs = {'axis': axis, 'center': center}
# vertices_flattend = net.flatten_network(method = method, **kwargs)

# method = "projection"
# kwargs = {'axis': axis}
# vertices_flattend = net.flatten_network(method = method, **kwargs)


We will only print the cable network from a single piece. Remove the struts:

In [ ]:
net.edges = np.delete(net.edges, strut_slice, axis=0)
net.q = np.delete(net.q, strut_slice, axis=0)
net.l1 = np.delete(net.l1, strut_slice, axis=0)

net.net_plot(plot_type = 'projection', color = True,vlabels = True, elables=False)

Open a material model:

In [ ]:
file_path = os.path.join('DATA', 'Avg_Stress_Strain_Overture_TPU.csv')
stress_data, strain_data = net.load_stress_strain_curve(file_path)
TPU_nl = {'stress':strain_data, 'strain': stress_data, 'v':0.3897, 'p':1.18e-9, 'A': 0.078294515, 'name': 'TPU Overture'} # TPU Overture non-conductive

# If all elements will get the same material, you can use the following line
A = [TPU_nl['A']]*len(net.edges)
l0, l_scalar = net.materialize_nonlinear(A, stress_data, strain_data, interpolation_kind = 'cubic')

Automatically resolve crossings:

In [ ]:
copied_vertices = net.account_for_crossings()
net.net_plot(plot_type = 'projection', color = True,vlabels = True, elables=False)
net._set_leafs()
net.l0[net.leaf_edges] -= 2 # Goal length are reduced here by 2mm to account for a connection loop.

Optimize vertex locations

In [ ]:
net.initialize_shape_optimizer(function_type = 'standard',  method = 'Gauss-Seidel',options ={"maxiter": 10000, "damping": .1, "tol": 1e-6})
net.optimize_vertices()
net.net_plot(color=True, plot_type='optimized', elables=False, vlabels = True)

In [ ]:
%matplotlib qt
ani = net.ShapeOptimizer.animate_optimization()

Scale the network and determine arc parameters. In this example almost no scaling needs to be applied, and the "arcs" are effectively straight lines:

In [ ]:
reference_point = [0,0,0]                         # The network will be scaler relative to this point
net.scale_vertices(reference_point, net.l_scalar, account_for_leafs = True) # If you don't provide a scalar, it will use network.l_scalar automatically

# # Determine the Radius and Angle of the circle that define the arc of each element
R, th = net.arc_param()
# # for the arc length (net.l1 * net.l_scalar) is used unless specified otherwise, for the cord length (net.l0) is used unless specified otherwise
xyz = net.arc_points()
net.net_plot(color=True, plot_type='arcs', elables=False, vlabels = True)


Account for intersections

In [ ]:
printing_params = load_params(r'DATA/NT_Eel_0.2mm.json')
net.jump_at_intersection(intersection_width = printing_params['d_nozzle']*1.5, intersection_height = printing_params['layer_height'], interpolation_function=None) 

Add connection loops

In [ ]:
alpha_loop = np.deg2rad(30) # The angle of the loop
L_loop  = 3.5                # The length of the loop
n_points = 60               # The number of points in the loop

start_loop_bools = [True, False, False]
end_loop_bools = [True, False, False]

net.all_loop_to_path(start_loop_bools, end_loop_bools, L_loop, alpha_loop, n_points)

L_real = L_loop + L_loop*np.tan(alpha_loop)
L_real

A running start can be added to the gcode 

In [ ]:
start_loop_bools = [False, True, True]
end_loop_bools = [False, True, False]
L_loop  = 1.5
n_points = 5
net.add_running_start(start_loop_bools, end_loop_bools, L_loop, n_points)

Add starter gcode

In [ ]:
printing_params = load_params(r'DATA/NT_Eel_0.2mm.json')
start_gcode     = open(r'DATA/start_gcode_Prusa.gcode', 'r').read()
end_gcode       = open(r'DATA/end_gcode_Prusa.gcode', 'r').read()

temperature_settings = {'first_layer_bed_temperature': 65, 'first_layer_temperature':205, 'K-factor': 0.20}
g_code = replace_brackets(start_gcode, temperature_settings)

comment = ''

bed_width = 230
bed_height = 210

gen = G_code_generator(printing_params=printing_params)

Add a purge line

In [ ]:
point0, point1 = [20,15,0.3], [90,15,0.3]
point2, point3 = [90,10,0.3], [20,10,0.3]
g_code += make_purge_line(printing_params, gen, point0, point1)
g_code += make_purge_line(printing_params, gen, point2, point3)

Loop trough the paths and add gcode

In [ ]:
for path_i, cor_list in enumerate(net.paths_xyz):
    cor_list[:,0] += bed_width/2  - 30
    cor_list[:,1] += bed_height/2 - 50
    g_code += add_path_to_gcode(cor_list, gen, printing_params, comment = 'Path (' + str(path_i) + ')')

g_code += end_gcode
save_path = os.path.join('DATA', 'generated_gcodes', model_name + '.gcode')
os.makedirs(os.path.dirname(save_path), exist_ok=True)
save_gcode(save_path, g_code)
print('G-code generated')